In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/24 13:19:15 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/24 13:19:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 13:19:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sparkDataset = spark.read.option('header', True).option('delimiter', ',').csv('Dataset.csv')
pandasDataset = pd.read_csv('Dataset.csv')

sparkDataset.show()
pandasDataset.head(10)

+----------+---------+---------+----------+
|article_id|author_id|viewer_id| view_date|
+----------+---------+---------+----------+
|         1|        3|        5|2019-08-01|
|         1|        3|        6|2019-08-02|
|         2|        7|        7|2019-08-01|
|         2|        7|        6|2019-08-02|
|         4|        7|        1|2019-07-22|
|         3|        4|        4|2019-07-21|
|         3|        4|        4|2019-07-21|
+----------+---------+---------+----------+



,article_id,author_id,viewer_id,view_date
0,1,3,5,2019-08-01
1,1,3,6,2019-08-02
2,2,7,7,2019-08-01
3,2,7,6,2019-08-02
4,4,7,1,2019-07-22
5,3,4,4,2019-07-21
6,3,4,4,2019-07-21


In [3]:
def article_views(views: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    views.createOrReplaceTempView('Views')
    sqlQuery = \
    '''
    SELECT DISTINCT(author_id) as id FROM Views
    WHERE author_id = viewer_id
    ORDER BY id
    '''
    output = spark.sql(sqlQuery = sqlQuery)
    return output

output = article_views(sparkDataset)
output.show()

+---+
| id|
+---+
|  4|
|  7|
+---+



In [4]:
def article_views(views: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    output = views.filter(views.author_id == views.viewer_id)\
                    .select(views.author_id.alias('id'))\
                    .distinct()\
                    .orderBy("author_id")
    return output

output = article_views(sparkDataset)
output.show()

+---+
| id|
+---+
|  4|
|  7|
+---+



In [5]:
def article_views(views: pd.DataFrame) -> pd.DataFrame:
    output = views[(views.author_id == views.viewer_id)].author_id.sort_values().unique()
    output = pd.DataFrame(output, columns = ['id'])
    return output

output = article_views(pandasDataset)
output.head()

,id
0,4
1,7
